In [5]:
import numpy as np

import os

from sentence_transformers import SentenceTransformer, util

from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd

import tensorflow as tf

In [6]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')#768维embedding

# sentences = ['I love UofT',
#             'I really love UofT', 
#             'I like UofT',
#             "I don't like UofT",
#              "I hate UofT",
#              "I like eating bread",
#              "I like earting bread in UofT",
#              "I want to study in UofT",
#              "I study in my home",
#              "educate in UofT"
#             ]
# embeddings = model.encode(sentences)

#Print the embeddings
# for sentence, embedding in zip(sentences, embeddings):
#     print("Sentence:", sentence)
#     print(embedding.shape)#768维embedding
#     print("Embedding:", embedding) 
#     print("")
   

In [7]:
df = pd.read_csv('./example.csv')

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [8]:
sentence_vecs = np.empty(shape=(768,len(df)))

sentence_vecs = np.row_stack((df['Publication number'],sentence_vecs))

In [9]:
for i in range(len(df)):
        
    sentence_vecs[1:,i]= model.encode(df['abstract'][i])#每一行768个槽位，encode后填入
    
#     print(i)

#### 此时第一列是全部的id， 右边的所有列都是一样的embeddings for abstract for each id

In [10]:
 #transpose后 69* 679   
sentence_vecs = sentence_vecs.T
sentence_vecs.shape

(69, 769)

In [11]:
# 查看此时表格长什么样？
pd.DataFrame(sentence_vecs)

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,768
0,CA2966757A1,0.001067,-0.054919,-0.003601,-0.031796,0.037003,0.02245,-0.078226,0.017043,0.015866,...,-0.010419,0.03925,0.014891,0.006084,-0.029682,-0.010392,-0.022623,-0.111363,-0.095344,-0.057547
1,CN104317959A,0.06156,-0.262383,-0.001671,0.068596,-0.140208,0.072899,0.166541,-0.121898,-0.062559,...,0.025093,-0.123495,-0.013164,-0.035926,-0.013122,0.476864,-0.11937,0.049146,-0.040324,-0.035843
2,CN108345630A,-0.014376,-0.285412,-0.001517,-0.005913,-0.023102,0.058965,0.313875,0.016019,0.077813,...,0.001334,-0.003237,0.127703,-0.085374,-0.049348,0.142474,-0.110853,0.002939,-0.096863,-0.001018
3,CN108197330A,0.136868,-0.219382,-0.002887,-0.050883,-0.114802,0.060588,0.065584,0.054701,0.055867,...,-0.036596,-0.019167,0.083532,-0.118996,0.016503,-0.191574,-0.078483,-0.023588,-0.173767,-0.113955
4,CN104317959B,0.013443,-0.156624,0.000975,-0.051676,-0.205327,0.139471,0.195919,-0.061621,0.097584,...,0.005217,-0.012723,-0.060217,-0.04056,0.03736,0.045731,-0.057188,0.061246,-0.171368,-0.030163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,MX380794B,0.174366,-0.342083,-0.003529,-0.062802,-0.140273,0.008996,0.073058,0.004762,-0.27643,...,0.097487,-0.025653,0.130515,-0.111905,-0.042269,0.278141,-0.07763,-0.02497,0.000569,0.020323
65,WO2021142369A1,-0.082103,-0.207149,0.00225,-0.02069,-0.074864,-0.0236,0.198786,-0.122726,-0.016218,...,-0.012216,-0.025499,0.234623,-0.079914,-0.053258,0.186293,-0.085498,0.031199,-0.14974,-0.05643
66,US11166031B2,-0.043905,0.002483,0.001484,0.004378,-0.046495,-0.082711,0.010052,-0.04595,-0.201746,...,0.02762,0.042335,0.107615,-0.044651,-0.010402,-0.014605,-0.08669,0.022204,-0.032164,-0.103435
67,CN306985016S,-0.122356,0.108768,0.004345,-0.053525,-0.131672,0.021583,0.130812,-0.052795,-0.048204,...,0.06048,-0.008072,0.018561,-0.070092,-0.055614,-0.009738,0.040804,-0.032736,-0.105464,-0.017578


#### 计算similarity最高的几个abstract

单独提取出abstract这一列变成list of string

In [12]:
# extract all abstracts
list_of_abstract = df["abstract"].to_list()

In [13]:
len(list_of_abstract)

69

查看是否有69个

In [14]:
#直接对list of string去算embeddings，是个长度为69的，每个里面768个
embeddings = model.encode(list_of_abstract)

In [15]:
print(len(embeddings))
print(len(embeddings[0]))
print(embeddings[0])#跟上面表格对应一样的，768维度的embedding

69
768
[ 1.06711301e-03 -5.49191348e-02 -3.60145932e-03 -3.17960270e-02
  3.70028205e-02  2.24502496e-02 -7.82263279e-02  1.70432962e-02
  1.58658288e-02  5.12934439e-02 -4.25125621e-02  1.61987171e-02
 -2.40938365e-02  3.27557415e-01 -3.30613442e-02  2.00313292e-02
  1.34672970e-02 -2.45091226e-02 -1.73476618e-02 -7.83115067e-03
  5.85978217e-02 -7.60765448e-02  5.97917698e-02  2.00123917e-02
  8.20848066e-03  5.30708283e-02  1.99628863e-02  3.42343114e-02
  5.72337955e-02  6.43678308e-02 -1.34059563e-02  9.87343714e-02
 -5.71312197e-03 -1.87257335e-01  1.20800167e-01 -5.21391854e-02
  2.89770160e-02  9.52979922e-02 -1.44641891e-01  2.29655057e-01
 -1.48534715e-01 -3.62751260e-02  2.04414735e-03  5.15988655e-02
  5.47474474e-02  1.12656698e-01  7.75335580e-02 -3.32655758e-02
  5.94144454e-03 -5.51085249e-02  1.06468787e-02 -9.79515463e-02
  1.02752812e-01  1.37089333e-02  1.06083922e-01 -6.99155265e-03
 -2.04339251e-02 -1.75420031e-01 -9.69166905e-02 -2.70758346e-02
  1.23017699e-01  

计算所有similarity， 但是由于会重复计算自己跟自己的，选取的时候要注意

In [16]:
#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1): #此处为了不重复提取similarity 为1的（自己跟自己比）
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})
        
print(pairs)

[{'index': [0, 1], 'score': tensor(0.1119)}, {'index': [0, 2], 'score': tensor(0.1161)}, {'index': [0, 3], 'score': tensor(0.2272)}, {'index': [0, 4], 'score': tensor(0.2158)}, {'index': [0, 5], 'score': tensor(0.2144)}, {'index': [0, 6], 'score': tensor(0.2393)}, {'index': [0, 7], 'score': tensor(0.0703)}, {'index': [0, 8], 'score': tensor(0.0609)}, {'index': [0, 9], 'score': tensor(0.2113)}, {'index': [0, 10], 'score': tensor(0.1583)}, {'index': [0, 11], 'score': tensor(0.0825)}, {'index': [0, 12], 'score': tensor(0.1909)}, {'index': [0, 13], 'score': tensor(0.1912)}, {'index': [0, 14], 'score': tensor(0.3141)}, {'index': [0, 15], 'score': tensor(0.2152)}, {'index': [0, 16], 'score': tensor(0.1547)}, {'index': [0, 17], 'score': tensor(0.2056)}, {'index': [0, 18], 'score': tensor(0.2067)}, {'index': [0, 19], 'score': tensor(0.2579)}, {'index': [0, 20], 'score': tensor(0.0433)}, {'index': [0, 21], 'score': tensor(0.2255)}, {'index': [0, 22], 'score': tensor(0.1669)}, {'index': [0, 23],

##### 看top10的

In [17]:
#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(list_of_abstract[i], list_of_abstract[j], pair['score']))

The invention relates to a trajectory planning system for a driverless car for a structured road, including a perception module, a positioning module, a lane change decision module, a motion planning module and a trajectory tracking module; the lane change decision module is based on the perception module and the positioning The data collected by the module outputs a decision action; the motion planning module outputs an optimal trajectory to the trajectory tracking module according to the decision action. The present invention uses deep reinforcement learning to make decisions. Based on the dynamic planning trajectory of decision actions, perception and control modules are independently processed. Compared with the end-to-end method, the explainability and operability of the decision planning process are greatly improved, which can be very It is well adapted to the system architecture of the previous driverless cars. 		 The invention discloses a method and a device for evaluating a dr